# 스타벅스 매장 데이터와 지도


* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

# 스타벅스 매장찾기
http://www.istarbucks.co.kr

* STORE >> 매장 찾기
* 지역검색 >> 서울 >> 전체

In [18]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
# %config InlineBackend.figure_format = 'retina'    # retina(망막) display: 망막으로 구분할 수 없다는 뜻의 애플 용어

In [19]:
"""
data = {
    'ins_lat':'37.56682', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'126.97865',
    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
"""


"\ndata = {\n    'ins_lat':'37.56682', # 지정한 위도와 경도에서 가까운 순으로 나열\n    'ins_lng':'126.97865',\n    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주\n    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)\n    'in_biz_cd':'',\n    'set_date':'',\n    'iend':'1000',\n}\n"

In [40]:
data = {
    'ins_lat':'', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'',
    'p_sido_cd':'03', # 01=서울시, 02=광주, 03=대구, 04:대전, 05=부산, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.starbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)    # request.get()과 동일, params 대신 data 사용.

In [41]:
print(r.text[:1000]) # 수신된 데이터의 앞부분만 확인

{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1573","s_name":"대구영대병원역DT","tel":"1522-3232","fax":"053-475-3748","sido_code":"03","sido_name":"대구","gugun_code":"0301","gugun_name":"남구","addr":"대구광역시 남구 봉덕동 595-7","park_info":null,"new_state":null,"theme_state":"T01@T04@T08@T09@T16@T17@T20@T21@T34@P80@P90","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":null,"all_search_cnt":-1,"addr_s

데이터는 JSON으로 수신되며, 형태는 다음과 같다.

```json
{
  "list": [
      {
            // ... 중략 ...
            "s_code": "1311",
            "s_name": "방화DT",
            "tel": "02-2664-3480",
            "fax": "02-2664-3481",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 방화동 293-4",
            "park_info": null,
            "new_state": null,
            "theme_state": "T17@T16@T09@T20@T01@T05@T08@T04",
            // ... 중략 ...
            "lat": "37.574339",
            "lot": "126.816415",
            "t22": 0
        },
        {
            // ... 중략 ...
            "s_code": "1267",
            "s_name": "마곡나루역",
            "tel": "02-3662-3504",
            "fax": "02-3662-3505",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 마곡동 759-3 보타닉파크타워Ⅰ105,203,204호",
            "park_info": null,
            "new_state": null,
            "theme_state": "T08@T05@T04@T17@T16@P80@T20",
            // ... 중략 ...
            "lat": "37.56813",
            "lot": "126.82614",
            "t22": 0
        },
    ]
 }
```

# JSON to DataFrame

json_normalize() 를 사용하여 JSON 데이터를 DataFrame로 전환 

In [42]:
jo = json.loads(r.text)

In [43]:
jo

{'list': [{'seq': 0,
   'sido_cd': None,
   'sido_nm': None,
   'gugun_cd': None,
   'gugun_nm': None,
   'code_order': None,
   'view_yn': None,
   'store_num': None,
   'sido': None,
   'gugun': None,
   'address': None,
   'new_img_nm': None,
   'p_pro_seq': 0,
   'p_view_yn': None,
   'p_sido_cd': '',
   'p_gugun_cd': '',
   'p_store_nm': None,
   'p_theme_cd': None,
   'p_wireless_yn': None,
   'p_smoking_yn': None,
   'p_book_yn': None,
   'p_music_yn': None,
   'p_terrace_yn': None,
   'p_table_yn': None,
   'p_takeout_yn': None,
   'p_parking_yn': None,
   'p_dollar_assent': None,
   'p_card_recharge': None,
   'p_subway_yn': None,
   'stb_store_file_renew': None,
   'stb_store_theme_renew': None,
   'stb_store_time_renew': None,
   'stb_store_lsm': None,
   's_code': '1573',
   's_name': '대구영대병원역DT',
   'tel': '1522-3232',
   'fax': '053-475-3748',
   'sido_code': '03',
   'sido_name': '대구',
   'gugun_code': '0301',
   'gugun_name': '남구',
   'addr': '대구광역시 남구 봉덕동 595-7',
   'p

In [44]:
type(jo)

dict

In [45]:
df = json_normalize(jo, 'list')

In [46]:
df

seq sido_cd sido_nm gugun_cd gugun_nm code_order view_yn store_num  sido  \
0     0    None    None     None     None       None    None      None  None   
1     0    None    None     None     None       None    None      None  None   
2     0    None    None     None     None       None    None      None  None   
3     0    None    None     None     None       None    None      None  None   
4     0    None    None     None     None       None    None      None  None   
..  ...     ...     ...      ...      ...        ...     ...       ...   ...   
61    0    None    None     None     None       None    None      None  None   
62    0    None    None     None     None       None    None      None  None   
63    0    None    None     None     None       None    None      None  None   
64    0    None    None     None     None       None    None      None  None   
65    0    None    None     None     None       None    None      None  None   

   gugun  ... p30 p70  p40 p80 t22 t21 p90 t05 t30 t36  
0   None  ...   0   0    0   0   0   0   0   0   0   0  
1   None  ...   0   0    0   0   0   0   0   0   0   0  
2   None  ...   0   0    0   0   0   0   0   0   0   0  
3   None  ...   0   0    0   0   0   0   0   0   0   0  
4   None  ...   0   0    0   0   0   0   0   0   0   0  
..   ...  ...  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  
61  None  ...   0   0    0   0   0   0   0   0   0   0  
62  None  ...   0   0    0   0   0   0   0   0   0   0  
63  None  ...   0   0    0   0   0   0   0   0   0   0  
64  None  ...   0   0    0   0   0   0   0   0   0   0  
65  None  ...   0   0    0   0   0   0   0   0   0   0  

[66 rows x 122 columns]

In [47]:
# 컬럼수 122개
df.columns

Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       'p30', 'p70', 'p40', 'p80', 't22', 't21', 'p90', 't05', 't30', 't36'],
      dtype='object', length=122)

In [48]:
# 주요한 컬럼 몇 가지 선택
               
df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df.head()

s_name        lat         lot sido_name gugun_name  \
0  대구영대병원역DT  35.845522  128.593536        대구         남구   
1     대구앞산DT  35.835516  128.579871        대구         남구   
2    계명대동산병원  35.853811  128.480276        대구        달서구   
3      대구죽전역  35.848437  128.536007        대구        달서구   
4     대구월곡DT  35.814866  128.541057        대구        달서구   

                 doro_address        tel  
0       대구광역시 남구 봉덕로 25 (봉덕동)  1522-3232  
1       대구광역시 남구 현충로 32 (대명동)  1522-3232  
2  대구광역시 달서구 달구벌대로 1035 (신당동)  1522-3232  
3          대구광역시 달서구 와룡로 169   1522-3232  
4     대구광역시 달서구 월곡로 225 (상인동)  1522-3232

# 위도 경도 데이터 타입 변환

In [49]:
df.dtypes

s_name          object
lat             object
lot             object
sido_name       object
gugun_name      object
doro_address    object
tel             object
dtype: object

### lat, lot타입으로 전환 (str→float) 

In [62]:
df['lat'] = df['lat'].astype(float)
df['lot'] = df['lot'].astype(float)

C:\Users\rta_note\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\rta_note\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
df.dtypes

s_name           object
lat             float64
lot             float64
sido_name        object
gugun_name       object
doro_address     object
tel              object
dtype: object

In [64]:
df.head(5)

s_name        lat         lot sido_name gugun_name  \
0  대구영대병원역DT  35.845522  128.593536        대구         남구   
1     대구앞산DT  35.835516  128.579871        대구         남구   
2    계명대동산병원  35.853811  128.480276        대구        달서구   
3      대구죽전역  35.848437  128.536007        대구        달서구   
4     대구월곡DT  35.814866  128.541057        대구        달서구   

                 doro_address        tel  
0       대구광역시 남구 봉덕로 25 (봉덕동)  1522-3232  
1       대구광역시 남구 현충로 32 (대명동)  1522-3232  
2  대구광역시 달서구 달구벌대로 1035 (신당동)  1522-3232  
3          대구광역시 달서구 와룡로 169   1522-3232  
4     대구광역시 달서구 월곡로 225 (상인동)  1522-3232

In [65]:
df.s_name

0     대구영대병원역DT
1        대구앞산DT
2       계명대동산병원
3         대구죽전역
4        대구월곡DT
        ...    
61       대구침산DT
62         대구칠곡
63          경북대
64     대구테크노폴리스
65         대구죽곡
Name: s_name, Length: 66, dtype: object

In [66]:
# (df.s_name == '아주대').sum()
(df.s_name == '경북대').sum()

1

### 특정 지점

In [67]:
# df[df['s_name'] == '시청'] # 시청점
df[df['s_name'] == '경북대'] 
# df[df['s_name'] == '부산대정문']   # 부산

s_name       lat         lot sido_name gugun_name           doro_address  \
63    경북대  35.89232  128.608586        대구         북구  대구광역시 북구 대학로 75 (산격동)   

          tel  
63  1522-3232

In [58]:
# df[df['s_name'] == '부산대정문'][['lat','lot']]

스타벅스 시청점의 위도(lat)와 경도(lot)는 각각  37.56629, 126.979808

# folium
https://github.com/python-visualization/folium

파이썬 지리정보 시각화 모듈  (문서: https://folium.readthedocs.io )

In [59]:
!pip install folium     

In [72]:
import folium         # python 지리정보 시각화 모듈

# 서울 시청 
# seoul_cityhall = (37.56629, 126.979808)
# ajou = (df[df['s_name'] == '아주대'].lat, df[df['s_name'] == '아주대'].lot)
kb = (df[df['s_name'] == '경북대'].lat, df[df['s_name'] == '경북대'].lot)

In [74]:
df[df['s_name'] == '경북대'].lat ,  df[df['s_name'] == '경북대'].lot

(63    35.89232
 Name: lat, dtype: float64, 63    128.608586
 Name: lot, dtype: float64)

In [75]:
# map_osm = folium.Map(location=seoul_cityhall)
map_osm = folium.Map(location = kb)
map_osm

In [77]:
map_osm = folium.Map(location = kb, zoom_start=17)
map_osm

In [78]:
map_osm = folium.Map(location = kb, zoom_start=17, tiles='Stamen Toner')
map_osm

In [80]:
map_osm = folium.Map(location=kb, zoom_start=17)
folium.Marker(kb, popup='경북대').add_to(map_osm)
folium.CircleMarker(kb, radius=50, color='#3186cc',
                   fill_color='#3186cc',).add_to(map_osm)
map_osm

In [82]:
# 전체 지점
map_osm = folium.Map(location=kb, zoom_start=11)

for ix, row in df.iterrows():        # Iterate over DataFrame rows as (index, Series) pair
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

지도를 상하좌우로 움직이거나 확대/축소 할 수 있다

In [27]:
# 지도의 확대값 지정 (zoom_start)
# map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17)
map_osm = folium.Map(location=ajou, zoom_start=17) # 아주대
map_osm

In [29]:
# 지도의 형태 변경.  tiles 인자 사용.
map_osm = folium.Map(location=ajou, zoom_start=17, tiles='Stamen Toner')
# map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17, tiles='Stamen Terrain')
map_osm

In [30]:
시청_좌표=(37.56629, 126.979808)
 
map_osm = folium.Map(location=ajou, zoom_start=17)
folium.Marker(ajou, popup='아주대').add_to(map_osm)
folium.CircleMarker(ajou, radius=50, color='#3186cc',
                   fill_color='#3186cc',).add_to(map_osm)
map_osm

# 서울 스타벅스 전 지점

In [79]:
help(df)

Help on DataFrame in module pandas.core.frame object:

class DataFrame(pandas.core.generic.NDFrame)
 |  Two-dimensional size-mutable, potentially heterogeneous tabular data
 |  structure with labeled axes (rows and columns). Arithmetic operations
 |  align on both row and column labels. Can be thought of as a dict-like
 |  container for Series objects. The primary pandas data structure.
 |  
 |  Parameters
 |  ----------
 |  data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
 |      Dict can contain Series, arrays, constants, or list-like objects
 |  
 |      .. versionchanged :: 0.23.0
 |         If data is a dict, argument order is maintained for Python 3.6
 |         and later.
 |  
 |  index : Index or array-like
 |      Index to use for resulting frame. Will default to RangeIndex if
 |      no indexing information part of input data and no index provided
 |  columns : Index or array-like
 |      Column labels to use for resulting frame. Will default to
 |    

In [80]:
map_osm = folium.Map(location=시청_좌표, zoom_start=11)

for ix, row in df.iterrows():        # Iterate over DataFrame rows as (index, Series) pair
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

# 제주도 스타벅스 매장 지도

In [26]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

한라산_좌표 = (33.361666, 126.529166)

data = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'16', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = json_normalize(json.loads(r.text), 'list')

map_osm = folium.Map(location=(33.361666, 126.529166), zoom_start=10)
for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

In [27]:
df[['s_name', 'gugun_name', 'doro_address', 'tel']]

s_name gugun_name                             doro_address           tel
0   제주신화월드R       서귀포시     제주특별자치도 서귀포시 안덕면 신화역사로304번길 38 B1-29  064-908-1729
1    제주에듀시티       서귀포시               제주특별자치도 서귀포시 대정읍 에듀시티로 36   064-792-3371
2    제주성산DT       서귀포시                 제주특별자치도 서귀포시 성산읍 일출로 80   064-782-3273
3    제주중문DT       서귀포시                제주특별자치도 서귀포시 천제연로 95(색달동)  064-738-8915
4     제주서귀포       서귀포시                제주특별자치도 서귀포시 중정로 53 (서귀동)  064-763-8707
5     제주송악산       서귀포시             제주특별자치도 서귀포시 형제해안로 322 (상모리)  064-792-8731
6     서귀포DT       서귀포시               제주특별자치도 서귀포시 일주서로 11 (강정동)  064-738-5943
7     성산일출봉       서귀포시         제주특별자치도 서귀포시 성산읍 일출로 284-5, 1,2층  064-782-8535
8      제주중문       서귀포시         제주특별자치도 서귀포시 중문관광로110번길 32 (색달동)  064-738-8510
9      제주시청        제주시               제주특별자치도 제주시 중앙로 215 (이도이동)  064-756-3511
10     제주함덕        제주시               제주특별자치도 제주시 조천읍 조함해안로 522   064-783-3537
11   제주노형공원        제주시                 제주특별자치도 제주시 연북로 12 (노형동)  064-746-3367
12   제주애월DT        제주시               제주특별자치도 제주시 애월읍 애월해안로 376   064-799-3273
13     제주칠성        제주시                제주특별자치도 제주시 관덕로 55 (일도일동)  064-757-9892
14     제주노형        제주시                 제주특별자치도 제주시 도령로 27 (노형동)  064-745-8914
15   제주용담DT        제주시       제주특별자치도 제주시 서해안로 380 (용담삼동) 화이트하우스  064-745-9817
16   신제주이마트        제주시  제주특별자치도 제주시 1100로 3348, 신제주이마트 1층 (노형동)  064-748-8657

## 부산지역

In [82]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

한라산_좌표 = (33.361666, 126.529166)

data = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'05', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = json_normalize(json.loads(r.text), 'list')

lat = df[df['s_name'] == '부산대정문'].lat
lot = df[df['s_name'] == '부산대정문'].lot

map_osm = folium.Map(location=(lat, lot), zoom_start=10)
map_osm

In [83]:
for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

# 요약
* requests.post() 데이터 가져오기
* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

### 2018 FinanceData.KR [fb.com/financedata](fb.com/financedata)